In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [6]:
url = 'https://nyheder.tv2.dk/seneste'
driver = webdriver.Chrome()
driver.get(url)

headers = {
    'Name': 'Mathias Udsen', 
    'E-mail':'lcv842@alumni.ku.dk',
    'Message': 'Scraping as a part of our exam project at UCPH, Intoduction to social data science' 
}
requests.get(url, headers)
time.sleep(5)

In [7]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()
time.sleep(5)

In [8]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()
time.sleep(5)

In [ ]:
nummer=0

max_retries = 30
retries = 0

Mathias = 'Ja'

max_iter = 500
i = 500

soup = BeautifulSoup(driver.page_source, 'html.parser')
count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))

if Mathias == 'Ja':
    
    pbar1 = tqdm(total=max_iter, desc="Loading content")

    while i < 1000 or retries < max_retries:
        retries = 0
        try:
            Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
            Vis_flere.click()

            new_count_of_one_column_lists = 0
            while new_count_of_one_column_lists <= count_of_one_column_lists:
                #soup = BeautifulSoup(driver.page_source, 'html.parser')
                soup = BeautifulSoup(driver.page_source, 'lxml')
                new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
                time.sleep(0.5)

            count_of_one_column_lists = new_count_of_one_column_lists
            i += 1
            nummer += 1
            pbar1.update(1)
        except:
            time.sleep(1)
            retries += 1
    pbar1.close()
time.sleep(5)

Loading content:   0%|          | 0/500 [00:00<?, ?it/s]


KeyboardInterrupt



In [4]:
"""
i = 0
count_of_one_column_lists = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
        i += 1
    except:
        time.sleep(1)
"""

"\ni = 0\ncount_of_one_column_lists = 0\nwhile i <= 10:\n    try:\n        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')\n        Vis_flere.click()\n\n        new_count_of_one_column_lists = 0\n        while new_count_of_one_column_lists <= count_of_one_column_lists:\n            soup = BeautifulSoup(driver.page_source, 'html.parser')\n            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))\n            time.sleep(0.25)\n\n        count_of_one_column_lists = new_count_of_one_column_lists\n        i += 1\n    except:\n        time.sleep(1)\n"

In [7]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
Live = []
All_text = []
Fejl = []
#nummer=0

Live_find1 = False
Live_find2 = False

if nummer == 500:
    max_iter = 1000
elif nummer == 0:
    max_iter = 500
else: 
    max_iter = nummer
    nummer = 0

pbar2 = tqdm(total=nummer, desc="Scraping")
i = 0
#for i in range(0, 500, 1):
for i in range(nummer, max_iter, 1):
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'lxml')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
    except:
        time.sleep(1)
        
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            #print(i)
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
        continue
        
    pbar3 = tqdm(total=len(soup.find_all('ul', {'data-columns': '1'})[i].find_all(class_='tc_teaser')), desc="Articles", leave=False)

    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')
            
        for artikel in artikler:
            nummer += 1

            # Initializations
            current_headline = None
            current_author = None
            current_time = None
            current_category = None
            current_text = []
            current_live = 0

            if artikel.find('span', class_='tc_label tc_label--color-live'):
                Live_find1 = True
            else:
                Live_find1 = False

            # Extract link
            href = artikel.find('a', class_='tc_teaser__link').get('href')
            
            try:
                if href:
                    driver.execute_script(f'window.open("{href}", "_blank");')
                    driver.switch_to.window(driver.window_handles[-1])

                    new_html = driver.page_source
                    new_soup = BeautifulSoup(new_html, 'lxml')

                    liveblog_body = new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context': "article-body"})
                    Live_find2 = bool(liveblog_body and liveblog_body.find('em') and liveblog_body.find('em').text == 'Denne liveblog opdateres ikke længere.')

                    if Live_find1 or Live_find2:
                        current_headline = new_soup.find('header', class_="tc_page__header").find('h1').text

                    byline = new_soup.find('div', class_="tc_byline")
                    if byline:
                        current_author = byline.find('strong').text
                    timestamp = new_soup.find('time', class_='tc_timestamp')
                    if timestamp:
                        current_time = timestamp.get('datetime')

                    if liveblog_body:
                        article_body_div = liveblog_body
                    else:
                        article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})

                    if article_body_div:
                        rich_content_paragraphs = article_body_div.find_all('p')
                        for paragraph in rich_content_paragraphs:
                            current_text.append(paragraph.text)

                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
                
            except Exception as e:
                print(f"Error processing article at nummer: {nummer}. Error: {e}. {href}")
                driver.close()   # Ensure the article page is closed even on error
                driver.switch_to.window(driver.window_handles[0])
                Fejl.append(href)
                continue

            if Live_find1 or Live_find2:
                current_live = 1
            else:
                current_headline = artikel.find('a', class_='tc_teaser__link').get('aria-label')

            label = artikel.find('span', class_='tc_label tc_label--color-nyheder')
            if label:
                current_category = label.text

            if all([current_headline, current_author, current_time, current_category, current_text]):
                Headline.append(current_headline)
                Authors.append(current_author)
                Time.append(current_time)
                Category.append(current_category)
                All_text.append(current_text)
                Live.append(current_live)
            else:
                print(f"Skipped article at nummer: {nummer} due to missing details: {href}")
                Fejl.append(href)
            pbar3.update(1)
    pbar3.close()
    pbar2.update(1)
pbar2.close()
time.sleep(5)

Loading content:   0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
print(len(Category), Category[0])
print(len(Authors), Authors[0])
print(len(Headline), Headline[0])
print(len(Time), Time[0])
print(len(Live), Live[0])
print(len(All_text), All_text[0])
time.sleep(5)

IndexError: list index out of range

In [8]:
data = {
    'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'Live': Live,
    'All_text': All_text
}
df = pd.DataFrame(data)
time.sleep(5)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
df
#sorted_df = df.sort_values(by='Time')
#sorted_df

,Category,Authors,Headline,Time,Live,All_text


In [ ]:
df.to_csv('Data.csv', index=False)
time.sleep(5)

In [ ]:
df.to_csv(r'C:\Users\Mathias\Desktop\Privat\Uni\~Kandidat\ISDS\Data.csv', index=False)
time.sleep(5)

In [ ]:
import os
os.system("shutdown /s /t 10")

In [ ]:
#os.system("shutdown /a")